# Solution of k-SAT formulas via BP + reinforcement


Draw a random 3-SAT formula and try to solve it using Belief Propagation. Reinforcement is necessary to break the symmetry :BP gives a uniform measure over all SAT assignments, we are only interested in finding one

In [12]:
using BeliefPropagation, BeliefPropagation.Models, Random

Draw a random formula with $N$ variables and $M$ clauses

In [14]:
rng = MersenneTwister(0)
n = 100
m = 100
k = 3
g = rand_regular_factor_graph(rng, n, m, k)
ψ = [KSATClause(bitrand(rng, length(neighbors(g, factor(a))))) for a in factors(g)]
bp = BP(g, ψ, fill(2, nvariables(g)));

Run BP

In [15]:
iters = iterate!(bp; maxiter=1000, tol=1e-12, rein=5e-2)

43

Given the beliefs $\{b_i\}_i$, our guess for the solving assignment is
$$ x_i^* = \arg\max_{x_i} b_i(x_i)$$

In [16]:
xstar = argmax.(beliefs(bp));

Check that the assignment satisfies the formula:

In [17]:
nunsat = sum(1 - Int(bp.ψ[a](xstar[i] for i in neighbors(bp.g, factor(a)))) for a in factors(bp.g))

0